In [15]:
import add_path
import datasets
import random
import json

d:\projects\affect\TUCORE-GCN\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from models.BERT.tucoregcn_bert_pytorch_processor import DialogRE
dev = DialogRE()
gen = dev._generate_examples("../datasets/DialogRE/dev.json", "dev")


In [3]:
test = gen.send(None)[1]
test

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'SpeakerBertTokenizer'.


SpeakerRelation(speaker_x='Estelle', speaker_y='Speaker 1', rid=[7])
Speaker 1, {speaker_y} Hey!
Speaker 1, Speaker 2: Hey.
Speaker 1, Speaker 3: Hey, man. What's up?
Speaker 1, {speaker_y} Maybe you can tell me. My agent would like to know why I didn't show up at the audition I didn't know I had today. The first good thing she gets me in weeks. How could you not give me the message?!
Speaker 1, Speaker 3: Well, I'll tell ya I do enjoy guilt, but, ah, it wasn't me.
Speaker 1, Speaker 2: Yes, it was! It was him! Uh huh! Okay, it was me!
Speaker 1, {speaker_y} How is it you?
Speaker 1, Speaker 2: Well, it was just, it was all so crazy, you know. I mean, Chandler was in the closet, counting to 10, and he was up to 7 and I hadn't found a place to hide yet. I-I-I meant to tell you, and I wrote it all down on my hand. See, all of it.
Speaker 1, {speaker_y} Yep, that's my audition.
Speaker 1, Speaker 4: See, now this is why I keep notepads everywhere.
Speaker 1, Speaker 2: Yep, and that's why

{'dialog': "{speaker_y} hey!\n{speaker_2} hey.\n{speaker_3} hey, man. what's up?\n{speaker_y} maybe you can tell me. my agent would like to know why i didn't show up at the audition i didn't know i had today. the first good thing she gets me in weeks. how could you not give me the message?!\n{speaker_3} well, i'll tell ya i do enjoy guilt, but, ah, it wasn't me.\n{speaker_2} yes, it was! it was him! uh huh! okay, it was me!\n{speaker_y} how is it you?\n{speaker_2} well, it was just, it was all so crazy, you know. i mean, chandler was in the closet, counting to 10, and he was up to 7 and i hadn't found a place to hide yet. i-i-i meant to tell you, and i wrote it all down on my hand. see, all of it.\n{speaker_y} yep, that's my audition.\n{speaker_4} see, now this is why i keep notepads everywhere.\n{speaker_2} yep, and that's why we don't invite you to play.\n{speaker_5} what is the great tragedy here? you go get yourself another appointment.\n{speaker_y} well, estelle tried, you know. t

In [18]:
from models.BERT.speaker_tokens import SPEAKER_TOKENS, SpeakerBertTokenizer

n_class = 36
speaker_tokenizer = SpeakerBertTokenizer.from_pretrained('bert-base-uncased')
speaker_tokens = [entry[1] for idx, entry in list(filter(lambda x: x[1][1] if not x[1][0].startswith("__") else None, enumerate(SPEAKER_TOKENS.__dict__.items())))]
sequence = speaker_tokenizer.tokenize(test['dialog'])
speaker_x = speaker_tokenizer.tokenize(test['relation'].speaker_x)
speaker_y = speaker_tokenizer.tokenize(test['relation'].speaker_y)

tokens = ["[CLS]"] + sequence + ["[SEP]"] + speaker_x + ["[SEP]"] + speaker_y + ["[SEP]"]
input_ids = speaker_tokenizer.convert_tokens_to_ids(tokens)
input_mask = [1] + [1]*len(sequence) + [1] + [1]*len(speaker_x) + [1] + [1]*len(speaker_y) + [0]
segment_ids = [0] + [0]*len(sequence) + [0] + [1]*len(speaker_x) + [1] + [1]*len(speaker_y) + [1]

input_speaker_ids = []
input_mention_ids = []
current_speaker_id = 0
current_speaker_idx = 0
for token in sequence:
	if SPEAKER_TOKENS.is_speaker(token):
		current_speaker_id = SPEAKER_TOKENS.convert_speaker_to_id(token)
		current_speaker_idx+=1
	input_speaker_ids.append(current_speaker_id)
	input_mention_ids.append(current_speaker_idx)
speaker_ids = [0] + input_speaker_ids + [0] + [0]*len(speaker_x) + [0] + [0]*len(speaker_y) + [0]
mention_ids = [0] + input_mention_ids + [0] + [0]*len(speaker_x) + [0] + [0]*len(speaker_y) + [0]

label_id = []
for k in range(n_class):
	if k+1 in test['relation'].rid:
		label_id.append(1)
	else:
		label_id.append(0)

print(f"tokens      length: {len(tokens)}, "+" ".join([str(token) for token in tokens]))
print(f"input_ids   length: {len(input_ids)}, "+" ".join([str(input) for input in input_ids]))
print(f"input_mask  length: {len(input_mask)}, "+" ".join([str(mask) for mask in input_mask]))
print(f"segment_ids length: {len(segment_ids)}, "+" ".join([str(segment) for segment in segment_ids]))
print(f"speaker_ids length: {len(speaker_ids)}, "+" ".join([str(speaker) for speaker in speaker_ids]))
print(f"mention_ids length: {len(mention_ids)}, "+" ".join([str(mention) for mention in mention_ids]))
print(f"label_id: {label_id}")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'SpeakerBertTokenizer'.


NameError: name 'test' is not defined

In [59]:
from models.BERT.speaker_tokens import SPEAKER_TOKENS, SpeakerBertTokenizer
from models.BERT.tucoregcn_bert_pytorch_processor import Conversation, SpeakerRelation, Message
import numpy as np
from collections import defaultdict
from itertools import permutations
import dgl

def make_speaker_infor(speaker_id, mention_id):
	tmp = defaultdict(set)
	for i in range(1, len(speaker_id)):
		if speaker_id[i] == 0:
			break
		tmp[speaker_id[i]].add(mention_id[i])
	
	speaker_infor = dict()
	for k, va in tmp.items():
		speaker_infor[k] = list(va)
	return speaker_infor

def make_entity_edges_infor(input_ids, mention_id):
	entity_edges_infor = {'h':[], 't':[]}
	head_mention_id = max(mention_id) - 1
	tail_mention_id = max(mention_id)
	head = list()
	tail = list()
	for i in range(len(mention_id)):
		if mention_id[i] == head_mention_id:
			head.append(input_ids[i])

	for i in range(len(mention_id)):
		if mention_id[i] == tail_mention_id:
			tail.append(input_ids[i])
	
	for i in range(len(input_ids) - len(head)):
		if input_ids[i:i+len(head)] == head:
			entity_edges_infor['h'].append(mention_id[i])
	
	for i in range(len(input_ids) - len(tail)):
		if input_ids[i:i+len(tail)] == tail:
			entity_edges_infor['t'].append(mention_id[i])
	
	return entity_edges_infor

def create_graph(speaker_infor, turn_node_num, entity_edges_infor, head_mention_id, tail_mention_id):
	d = defaultdict(list)
	used_mention = set()

	# add speaker edges
	for _, mentions in speaker_infor.items():
		for h, t in permutations(mentions, 2):
			d[('node', 'speaker', 'node')].append((h, t))
			used_mention.add(h)
			used_mention.add(t)
	
	if d[('node', 'speaker', 'node')] == []:
		d[('node', 'speaker', 'node')].append((1, 0))
		used_mention.add(1)
		used_mention.add(0)
	

	# add dialog edges
	for i in range(1, turn_node_num+1):
		d[('node', 'dialog', 'node')].append((i, 0))
		d[('node', 'dialog', 'node')].append((0, i))
		used_mention.add(i)
		used_mention.add(0)
	if d[('node', 'dialog', 'node')] == []:
		d[('node', 'dialog', 'node')].append((1, 0))
		used_mention.add(1)
		used_mention.add(0)

	# add entity edges
	for mention in entity_edges_infor['h']:
		d[('node', 'entity', 'node')].append((head_mention_id, mention))
		d[('node', 'entity', 'node')].append((mention, head_mention_id))
		used_mention.add(head_mention_id)
		used_mention.add(mention)
	
	for mention in entity_edges_infor['t']:
		d[('node', 'entity', 'node')].append((tail_mention_id, mention))
		d[('node', 'entity', 'node')].append((mention, tail_mention_id))
		used_mention.add(tail_mention_id)
		used_mention.add(mention)
	
	if entity_edges_infor['h'] == []:
		d[('node', 'entity', 'node')].append((head_mention_id, 0))
		d[('node', 'entity', 'node')].append((0, head_mention_id))
		used_mention.add(head_mention_id)
		used_mention.add(0)
	
	if entity_edges_infor['t'] == []:
		d[('node', 'entity', 'node')].append((tail_mention_id, 0))
		d[('node', 'entity', 'node')].append((0, tail_mention_id))
		used_mention.add(tail_mention_id)
		used_mention.add(0)

	graph = dgl.heterograph(d)

	return graph, used_mention


def mention2mask(mention_id, old_behaviour=False):
    slen = len(mention_id)
    mask = []
    if old_behaviour:
        turn_mention_ids = [i for i in range(1, np.max(mention_id)-1)] #-1
    else:
        turn_mention_ids = [i for i in range(1, np.max(mention_id)+1)] #-1
    print(turn_mention_ids)
    for j in range(slen):
        tmp = None
        if mention_id[j] not in turn_mention_ids:
            tmp = np.zeros(slen, dtype=bool)
            if old_behaviour:
                tmp[j] = 1
        else:
            start = mention_id[j]
            end = mention_id[j]
            if mention_id[j] - 1 in turn_mention_ids:
                start = mention_id[j] - 1

            if mention_id[j] + 1 in turn_mention_ids:
                end = mention_id[j] + 1
            
            tmp = (mention_id >= start) & (mention_id <= end)
        mask.append(tmp)
    mask = np.stack(mask)
    return mask

n_class = 36
max_seq_length=512
speaker_tokenizer = SpeakerBertTokenizer.from_pretrained('bert-base-uncased')
c = Conversation(
	messages=[
		Message("Speaker 1", "Howdy! I'm Flowey, Flowey the Flower!"),
		Message("Speaker 2", "Hello Flowey."),
	],
	speaker_relations=[
		SpeakerRelation("Speaker 1", "Speaker 2", [3])
	]
)
old_behaviour = True
test2 = c.build_inputs(speaker_tokenizer)[0]

speaker_tokens = [entry[1] for idx, entry in list(filter(lambda x: x[1][1] if not x[1][0].startswith("__") else None, enumerate(SPEAKER_TOKENS.__dict__.items())))]
sequence = speaker_tokenizer.tokenize(test2['dialog'])
speaker_x = speaker_tokenizer.tokenize(test2['relation'].speaker_x)
speaker_y = speaker_tokenizer.tokenize(test2['relation'].speaker_y)

tokens = ["[CLS]"] + sequence + ["[SEP]"] + speaker_x + ["[SEP]"] + speaker_y + ["[SEP]"]
input_ids = speaker_tokenizer.convert_tokens_to_ids(tokens)
input_mask = [1] + [1]*len(sequence) + [1] + [1]*len(speaker_x) + [1] + [1]*len(speaker_y) + [0]
segment_ids = [0] + [0]*len(sequence) + [0] + [1]*len(speaker_x) + [1] + [1]*len(speaker_y) + [1]

input_speaker_ids = []
input_mention_ids = []
current_speaker_id = 0
current_speaker_idx = 0
for token in sequence:
	if SPEAKER_TOKENS.is_speaker(token):
		current_speaker_id = SPEAKER_TOKENS.convert_speaker_to_id(token)
		current_speaker_idx+=1
	input_speaker_ids.append(current_speaker_id)
	input_mention_ids.append(current_speaker_idx)
if old_behaviour:
	speaker_ids = [0] + input_speaker_ids + [0] + [SPEAKER_TOKENS.convert_speaker_to_id(test2['relation'].speaker_x)]*len(speaker_x) + [0] + [SPEAKER_TOKENS.convert_speaker_to_id(test2['relation'].speaker_y)]*len(speaker_y) + [0]
	mention_ids = [0] + input_mention_ids + [0] + [current_speaker_idx+1]*len(speaker_x) + [0] + [current_speaker_idx+2]*len(speaker_y) + [0]
else:
	speaker_ids = [0] + input_speaker_ids + [0] + [0]*len(speaker_x) + [0] + [0]*len(speaker_y) + [0]
	mention_ids = [0] + input_mention_ids + [0] + [0]*len(speaker_x) + [0] + [0]*len(speaker_y) + [0]
label_id = []
for k in range(n_class):
	if k+1 in test2['relation'].rid:
		label_id.append(1)
	else:
		label_id.append(0)

while len(input_ids) < max_seq_length:
	tokens.append('[PAD]')
	input_ids.append(0)
	input_mask.append(0)
	segment_ids.append(0)
	speaker_ids.append(0)
	mention_ids.append(0)

turn_masks = mention2mask(np.array(mention_ids),old_behaviour)

speaker_infor = make_speaker_infor(speaker_ids, mention_ids)
turn_node_num = max(mention_ids) - 2
head_mention_id = max(mention_ids) - 1
tail_mention_id = max(mention_ids)
entity_edges_infor = make_entity_edges_infor(input_ids, mention_ids)
graph, used_mention = create_graph(speaker_infor, turn_node_num, entity_edges_infor, head_mention_id, tail_mention_id)
assert len(used_mention) == (max(mention_ids) + 1)

print(f"label_id: {label_id}")
print(f"tokens        length: {len(tokens)}, "+" ".join([str(token) for token in tokens]))
print(f"input_ids     length: {len(input_ids)}, "+" ".join([str(input) for input in input_ids]))
print(f"input_mask    length: {len(input_mask)}, "+" ".join([str(mask) for mask in input_mask]))
print(f"segment_ids   length: {len(segment_ids)}, "+" ".join([str(segment) for segment in segment_ids]))
print(f"speaker_ids   length: {len(speaker_ids)}, "+" ".join([str(speaker) for speaker in speaker_ids]))
print(f"mention_ids   length: {len(mention_ids)}, "+" ".join([str(mention) for mention in mention_ids]))
print(f"turn_masks[0] length: {len(turn_masks)}, "+" ".join([str(turn_mask) for turn_mask in turn_masks[0]]))
print(f"graph:"+str(graph))
print(entity_edges_infor)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'SpeakerBertTokenizer'.


[1, 2]
label_id: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
tokens        length: 512, [CLS] {speaker_x} how ##dy ! i ' m flow ##ey , flow ##ey the flower ! {speaker_y} hello flow ##ey . [SEP] {speaker_x} [SEP] {speaker_y} [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

In [56]:
speaker_x[0]

'{speaker_x}'

In [55]:
[entry[1] for idx, entry in list(filter(lambda x: x[1][1] if not x[1][0].startswith("__") else None, enumerate(SPEAKER_TOKENS.__dict__.items())))]

['{speaker_1}',
 '{speaker_2}',
 '{speaker_3}',
 '{speaker_4}',
 '{speaker_5}',
 '{speaker_6}',
 '{speaker_7}',
 '{speaker_8}',
 '{speaker_9}',
 '{speaker_x}',
 '{speaker_y}',
 <function models.BERT.speaker_tokens.SPEAKER_TOKENS.is_speaker(token)>,
 <function models.BERT.speaker_tokens.SPEAKER_TOKENS.convert_speaker_to_id(token)>]

In [8]:
[entry[1] for idx, entry in list(filter(lambda x: x[1][1] if not (x[1][0].startswith("__") or type(x[1][1])!=str) else None, enumerate(SPEAKER_TOKENS.__dict__.items())))].index("{speaker_1}")

0

In [9]:
type(list(SPEAKER_TOKENS.__dict__.items())[12][1])

function